# Simulation

- In this lesson, we will work through several examples of using random numbers to simulate real-world scenarios.
- For reference, the [viz module](./viz.py) contains the visuals used for these lessons.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import viz # curriculum example visualizations

np.random.seed(29)

## Generating Random Numbers with Numpy

The `numpy.random` module provides a number of functions for generating random numbers.

- `np.random.choice`: selects random options from a list
- `np.random.random`: generates numbers between 0 and 1
- `np.random.uniform`: generates numbers between a given lower and upper bound
- `np.random.randn`: generates numbers from the standard normal distribution
- `np.random.normal`: generates numbers from a normal distribution with a specified mean and standard deviation

## Example Problems

### Carnival Dice Rolls

> You are at a carnival and come across a person in a booth offering you a game
> of "chance" (as people in booths at carnivals tend to do).

> You pay **5 dollars** and roll 3 dice. If the sum of the dice rolls is greater
> than 12, you get **15 dollars**. If it's less than or equal to 12, you get
> **nothing**.

> Assuming the dice are fair, should you play this game? How would this change
> if the winning condition was a sum greater than *or equal to* 12?

To simulate this problem, we'll write the python code to simulate the scenario described above, then repeat it a large amount of times.

One way we can keep track of all the simulations is to use a 2-dimensional matrix. We can create a matrix where each row represents one "trial". Each row will have 3 columns, representing the 3 dice rolls.

Let see how `np.random.choice()` can help us with this question

[np.random.choice() documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)

In [ ]:
n_trials = nrows = 10_000
n_dice = ncols = 3

rolls = np.random.choice([1, 2, 3, 4, 5, 6], size = (n_trials, n_dice))
rolls

Here we used the `choice` function to randomly select an element out of the list of the number 1-6, effectively simulating a dice roll. The second argument supplied to `choice` is the total number of dice to roll. 

Now that we have all of the simulated dice rolls, we want to get the sum of the dice rolls for each trial. To do this, we can use the `.sum` function and specify that we want the sum of every row (as opposed to the sum of all the numbers, or the sum by column) with the `axis` key word argument.

In [ ]:
sums_by_trial = rolls.sum(axis=1)
sums_by_trial

Let's pause here for a minute and visualize the data we have:

In [ ]:
viz.simulation_example1(sums_by_trial)

The area shaded in lightblue represents our chance of winning, that is, the number of times that the sum of 3 dice rolls is greater than 12.

We can now convert each value in our array to a boolean value indicating whether or not we won:

In [ ]:
wins = sums_by_trial > 12
wins

To calculate an overall win rate, we can treat each win as a `1` and each loss as `0`, then take the average of the array:

In [ ]:
win_rate = wins.mean()
win_rate

Now that we know our win rate, we can calculate the expected profit:

In [ ]:
expected_winnings = win_rate * 15
cost = 5
expected_profit = expected_winnings - cost
expected_profit

So we would expect, based on our simulations, on average, to lose a little over a dollar everytime we play this game.

**How would this change if the winning condition was a sum greater than or equal to 12?**

We can recalculate our win rate based on the sums being greater than or equal to 12:

In [ ]:
wins = sums_by_trial >= 12
wins

In [ ]:
win_rate = wins.mean()
win_rate

In [ ]:
expected_winnings = win_rate * 15
expected_winnings

In [ ]:
cost = 5
expected_profit = expected_winnings - cost
expected_profit

If our win condition changes to the sum being greater than or equal to 12, then, based on our simulations, on average, we expect to win about 58 cents.

### No Rest or Relaxation

> There's a 30% chance my toddler takes a nap on any given weekend day. What is the chance that he takes a nap at least one day this weekend? What is the probability that he doesn't nap at all?

Let's first do a little bit of setup:

In [ ]:
p_nap = .3
ndays = ncols = 2
n_simulated_weekends = nrows = 10**5

To simulate the results from many weekends, we'll create a 2 x 10,000 matrix, with 2 being the number of days in a weekend and 10,000 being the number of simulations we want to run.

To determine whether or not a nap is taken on a given day, we'll generate a random number between 0 and 1, and say that it is a nap if it is less than our probability of taking a nap.

To do this, we are going to use `np.random.random()`

[np.random.random() documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.random.html)

In [ ]:
data = np.random.random(size =(nrows, ncols))
data

In [ ]:
naps = data < p_nap
naps

Now that we have each day as either true or false, we can take the sum of each row to find the total number of naps for the weekend. When we sum an array of boolean values, numpy will treat `True` as 1 and `False` as 0.

In [ ]:
naps.sum(axis=1)

Now we have the results of our simulation, an array where each number in the array represents how many naps were taken in a two day weekend.

In [ ]:
viz.simulation_example2(naps)

We can use this to answer our original questions, what is the probability that at least one nap is taken?

In [ ]:
(naps.sum(axis=1) >= 1).mean()

What is the probability no naps are taken?

In [ ]:
(naps.sum(axis=1) == 0).mean()

### Using Dataframes

Let's take a look at one more problem:

> What is the probability of getting at least one 3 in 3 dice rolls?

To simulate this, we'll use a similar strategy to how we modeled the dice rolls in the previous example, but this time, we'll store the results in a pandas dataframe so that we can apply a lambda function that will check to see if one of the rolls was a 3.

In [ ]:
n_simulations = nrows = 10**5
n_dice_rolled = ncols = 3

rolls = np.random.choice([1, 2, 3, 4, 5, 6], size = (nrows, ncols))
rolls

In [ ]:
rolls_df = pd.DataFrame(rolls, columns=['roll_1', 'roll_2', 'roll_3'])
rolls_df.head()

In [ ]:
rolls_df['has_3'] = rolls_df.apply(lambda row: 3 in row.values, axis=1)
rolls_df.head()

In [ ]:
rolls_df.has_3.mean()

In [ ]:
(pd.DataFrame(rolls)
 .apply(lambda row: 3 in row.values, axis=1)
 .mean())

Break down of what's going on here:

1. First we assign values for the number of rows and columns we are going to use
1. Next we create the `rolls` variable that holds a 3 x 10,000 matrix where each element is a randomly chosen number from 1 to 6
1. Lastly we create a dataframe from the rolls
    1. `pd.DataFrame(rolls)` converts our 2d numpy matrix to a pandas DataFrame
    1. `.apply(...` applies a function to each **row** in our dataframe, because we specified `axis=1`, the function will be called with each row as it's argument. The body of the function checks to see if the value `3` is in the values of the row, and will return either `True` or `False`
    1. `.mean()` takes our resulting series of boolean values, and treats `True` as 1 and `False` as 0, to give us the average rate of `True`s, in this case, the simulated probability of getting a 3 in 3 dice rolls.

We stil have three other commonly used random functions that we haven't practiced yet:

- `np.random.uniform`: generates numbers between a given lower and upper bound
- `np.random.randn`: generates numbers from the standard normal distribution
- `np.random.normal`: generates numbers from a normal distribution with a specified mean and standard deviation

Take some time to review the documentation of each, then start working on the exercises:

# Simulation Exercises

Using the repo setup directions, setup a new local and remote repository named `statistics-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `statistics-exercises`.

Do your work for this exercise in either a python file named `simulation.py` or a jupyter notebook named `simulation.ipynb`.

1. How likely is it that you roll doubles when rolling two dice?


2. If you flip 8 coins, what is the probability of getting exactly 3 heads? What is the probability of getting more than 3 heads?


3. There are approximately 3 web development cohorts for every 1 data science cohort at Codeup. Assuming that Codeup randomly selects an alumni to put on a billboard, what are the odds that the two billboards I drive past both have data science students on them?


4. Codeup students buy, on average, 3 poptart packages with a standard deviation of 1.5 a day from the snack vending machine. If on monday the machine is restocked with 17 poptart packages, how likely is it that I will be able to buy some poptarts on Friday afternoon? (Remember, if you have mean and standard deviation, use the np.random.normal) You'll need to make a judgement call on how to handle some of your values.


5. Compare Heights

- Men have an average height of 178 cm and standard deviation of 8cm.
- Women have a mean of 170, sd = 6cm.
- Since you have means and standard deviations, you can use `np.random.normal` to generate observations.
- If a man and woman are chosen at random, what is the likelihood the woman is taller than the man?

6. When installing anaconda on a student's computer, there's a 1 in 250 chance that the download is corrupted and the installation fails. 
- What are the odds that after having 50 students download anaconda, no one has an installation issue? 100 students?
- What is the probability that we observe an installation issue within the first 150 students that download anaconda?
- How likely is it that 450 students all download anaconda without an issue?

7. There's a 70% chance on any given day that there will be at least one food truck at Travis Park. 
- However, you haven't seen a food truck there in 3 days. How unlikely is this?
- How likely is it that a food truck will show up sometime this week?

8. If 23 people are in the same room, what are the odds that two of them share a birthday? What if it's 20 people? 40 people?

Be sure to add, commit, and push your work. Additional bonus exercises are available on the curriculum website. 